## Introduction

There are many powerful tools and packages in **R**. I have been spoiled by the convenient tools for interpretation. For example, the `lm` package provides readable summary output and easy to use formula as input for the model. However, things are not as easy and out of box in the `python` world. Although the machine learning packages such as `sklearn` and `tensoflow` are mature, things can be tricky when we deal with linear regression and categorical variable.

Recently, I have opportunity to work with a company for my capstone project. The company has strong preference for python language. One of the main objectives of the project is interpretibility. The golden rule of interpretibility is to avoid the black box machine learning model such as neural network. Our group decide to use logistic regression for our project. Being familiar with *R* package and the conveniency of use of formula and under-the-hood of handling of categorical variables, we were surprised of how complicated things can be with python. Given that pandas pacakge come with its `get_dummy` function, it comes with the imfamous [dummy trap](https://www.algosome.com/articles/dummy-variable-trap-regression.html).

Thanksfully, [Statsmodel](https://www.statsmodels.org/stable/index.html) comes in for the rescue. Statsmodel is a Python module that provides classes and functions for the estimation of many different statistical models. It also support *R*-style formula. 


In [4]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [5]:
# load data 
dat = sm.datasets.get_rdataset("Guerry", "HistData").data

Let's take a pick at our data!

In [6]:
dat.head()

,dept,Region,Department,Crime_pers,Crime_prop,Literacy,Donations,Infants,Suicides,MainCity,...,Crime_parents,Infanticide,Donation_clergy,Lottery,Desertion,Instruction,Prostitutes,Distance,Area,Pop1831
0,1,E,Ain,28870,15890,37,5098,33120,35039,2:Med,...,71,60,69,41,55,46,13,218.372,5762,346.03
1,2,N,Aisne,26226,5521,51,8901,14572,12831,2:Med,...,4,82,36,38,82,24,327,65.945,7369,513.00
2,3,C,Allier,26747,7925,13,10973,17044,114121,2:Med,...,46,42,76,66,16,85,34,161.927,7340,298.26
3,4,E,Basses-Alpes,12935,7289,46,2733,23018,14238,1:Sm,...,70,12,37,80,32,29,2,351.399,6925,155.90
4,5,E,Hautes-Alpes,17488,8174,69,6962,23076,16171,1:Sm,...,22,23,64,79,35,7,1,320.280,5549,129.10


We can use `R`-style formula with the new api from stastsmodel!

In [7]:
results = smf.ols('Lottery ~ Literacy + np.log(Pop1831)', data=dat).fit()

One of the great advantage of statsmodel is that it provide readable summary report for the models. 

In [8]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                Lottery   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.333
Method:                 Least Squares   F-statistic:                     22.20
Date:                Mon, 27 May 2019   Prob (F-statistic):           1.90e-08
Time:                        12:27:54   Log-Likelihood:                -379.82
No. Observations:                  86   AIC:                             765.6
Df Residuals:                      83   BIC:                             773.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         246.4341     35.233     

However, there ares still some short coming from the package. For example, the current implementation of **Logistic Regression** does not support weight for the variables. Although **glm** does have `freq_weight` and other parameters, they are not well tested and documented. This can be troublesome when we are dealing with imbalanced data. 

A good way to solve this problem is to use **sklearn** package. What if we can use R-style formula with sklearn?

The answer is **YES, WE CAN**



under the hood of statsmodel, the formula is handled by the patsy, which is the package which handles the formula. 

We can actually use patsy api to return the dataframe X and y for the sklearn model from our data

The function we will be using is [dmatrices](https://patsy.readthedocs.io/en/latest/API-reference.html#patsy.dmatrices). We will use "dataframe" for our `return_type`

In [12]:
from patsy import dmatrices, dmatrix, demo_data

y, X = dmatrices("Lottery ~ Literacy + np.log(Pop1831)", data = dat, return_type= "dataframe")

In [15]:
X.head()

,Intercept,Literacy,np.log(Pop1831)
0,1.0,37.0,5.846525
1,1.0,51.0,6.240276
2,1.0,13.0,5.697966
3,1.0,46.0,5.049215
4,1.0,69.0,4.860587


In [16]:
y.head()

,Lottery
0,41.0
1,38.0
2,66.0
3,80.0
4,79.0


The beauty of the patsy and the formula is that it is much more readable and it can support dummy encoding right from the formula. Let's say we are adding the categorical variable,`department` in our formula and all we need to do is to enclose Department with `C()` (**C** stands for categorical)

In [19]:
y, X = dmatrices("Lottery ~ Literacy + np.log(Pop1831) +  C(Department)", data = dat, return_type= "dataframe")

In [20]:
X.head()

,Intercept,C(Department)[T.Aisne],C(Department)[T.Allier],C(Department)[T.Ardeche],C(Department)[T.Ardennes],C(Department)[T.Ariege],C(Department)[T.Aube],C(Department)[T.Aude],C(Department)[T.Aveyron],C(Department)[T.Bas-Rhin],...,C(Department)[T.Tarn],C(Department)[T.Tarn-et-Garonne],C(Department)[T.Var],C(Department)[T.Vaucluse],C(Department)[T.Vendee],C(Department)[T.Vienne],C(Department)[T.Vosges],C(Department)[T.Yonne],Literacy,np.log(Pop1831)
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,5.846525
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.0,6.240276
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,5.697966
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,5.049215
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.0,4.860587


In [22]:
list(X.columns.values)

['Intercept',
 'C(Department)[T.Aisne]',
 'C(Department)[T.Allier]',
 'C(Department)[T.Ardeche]',
 'C(Department)[T.Ardennes]',
 'C(Department)[T.Ariege]',
 'C(Department)[T.Aube]',
 'C(Department)[T.Aude]',
 'C(Department)[T.Aveyron]',
 'C(Department)[T.Bas-Rhin]',
 'C(Department)[T.Basses-Alpes]',
 'C(Department)[T.Basses-Pyrenees]',
 'C(Department)[T.Bouches-du-Rhone]',
 'C(Department)[T.Calvados]',
 'C(Department)[T.Cantal]',
 'C(Department)[T.Charente]',
 'C(Department)[T.Charente-Inferieure]',
 'C(Department)[T.Cher]',
 'C(Department)[T.Correze]',
 'C(Department)[T.Corse]',
 "C(Department)[T.Cote-d'Or]",
 'C(Department)[T.Cotes-du-Nord]',
 'C(Department)[T.Creuse]',
 'C(Department)[T.Deux-Sevres]',
 'C(Department)[T.Dordogne]',
 'C(Department)[T.Doubs]',
 'C(Department)[T.Drome]',
 'C(Department)[T.Eure]',
 'C(Department)[T.Eure-et-Loir]',
 'C(Department)[T.Finistere]',
 'C(Department)[T.Gard]',
 'C(Department)[T.Gers]',
 'C(Department)[T.Gironde]',
 'C(Department)[T.Haut-Rhin]',

By default, the `C(Department)` is the same as `C(Department, Treatment)` which is the dummy encoding for categorical variables and by default it will drop the first value to avoid dummy trap. Patsy also support effect encoding which we will go over in future post. 

Another advantage of patsy formula is it also support various effect coding for the categorical variable
